# 贝叶斯分类器（伯努利模型）
  

In [4]:
import re
import numpy as np
import pandas as pd
import os
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer as ss
import chardet
import codecs

## 给20类文本集打上标签，并遍历所有文本文件

In [5]:
labels={0:'G:/PYCHARM/untitled3/20news-18828/alt.atheism',
       1:'G:/PYCHARM/untitled3/20news-18828/comp.graphics',
        2:'G:/PYCHARM/untitled3/20news-18828/comp.os.ms-windows.misc',
        3:'G:/PYCHARM/untitled3/20news-18828/comp.sys.ibm.pc.hardware',
        4:'G:/PYCHARM/untitled3/20news-18828/comp.sys.mac.hardware',
        5:'G:/PYCHARM/untitled3/20news-18828/comp.windows.x',
        6:'G:/PYCHARM/untitled3/20news-18828/misc.forsale',
        7:'G:/PYCHARM/untitled3/20news-18828/rec.autos',
        8:'G:/PYCHARM/untitled3/20news-18828/rec.motorcycles',
        9:'G:/PYCHARM/untitled3/20news-18828/rec.sport.baseball',
        10:'G:/PYCHARM/untitled3/20news-18828/rec.sport.hockey',
        11:'G:/PYCHARM/untitled3/20news-18828/sci.crypt',
        12:'G:/PYCHARM/untitled3/20news-18828/sci.electronics',
        13:'G:/PYCHARM/untitled3/20news-18828/sci.med',
        14:'G:/PYCHARM/untitled3/20news-18828/sci.space',
        15:'G:/PYCHARM/untitled3/20news-18828/soc.religion.christian',
        16:'G:/PYCHARM/untitled3/20news-18828/talk.politics.guns',
        17:'G:/PYCHARM/untitled3/20news-18828/talk.politics.mideast',
        18:'G:/PYCHARM/untitled3/20news-18828/talk.politics.misc',
        19:'G:/PYCHARM/untitled3/20news-18828/talk.religion.misc'
       }

In [6]:
path='G:/PYCHARM/untitled3/20news-18828'
packs=os.listdir(path)
result2=[]
for pack in packs:
     path1=path+"/"+pack
     print(path1)
     files=os.listdir(path1)
     result1.append(path1)
     for file in files:
          path2=path1+"/"+file
          result2.append(path2)

print('文档数:',len(result2))

G:/PYCHARM/untitled3/20news-18828/alt.atheism
G:/PYCHARM/untitled3/20news-18828/comp.graphics
G:/PYCHARM/untitled3/20news-18828/comp.os.ms-windows.misc
G:/PYCHARM/untitled3/20news-18828/comp.sys.ibm.pc.hardware
G:/PYCHARM/untitled3/20news-18828/comp.sys.mac.hardware
G:/PYCHARM/untitled3/20news-18828/comp.windows.x
G:/PYCHARM/untitled3/20news-18828/misc.forsale
G:/PYCHARM/untitled3/20news-18828/rec.autos
G:/PYCHARM/untitled3/20news-18828/rec.motorcycles
G:/PYCHARM/untitled3/20news-18828/rec.sport.baseball
G:/PYCHARM/untitled3/20news-18828/rec.sport.hockey
G:/PYCHARM/untitled3/20news-18828/sci.crypt
G:/PYCHARM/untitled3/20news-18828/sci.electronics
G:/PYCHARM/untitled3/20news-18828/sci.med
G:/PYCHARM/untitled3/20news-18828/sci.space
G:/PYCHARM/untitled3/20news-18828/soc.religion.christian
G:/PYCHARM/untitled3/20news-18828/talk.politics.guns
G:/PYCHARM/untitled3/20news-18828/talk.politics.mideast
G:/PYCHARM/untitled3/20news-18828/talk.politics.misc
G:/PYCHARM/untitled3/20news-18828/talk.r

   
   
   仿照之前vsm的操作，对每个class下的每个文本进行去标点、stem、去停用词的操作，每处理完一个class就将其中词频过低以及过高的词都删减掉，并将处理后剩下的词存入与该class对应的vocab[key]中，同时vocab_deput用来存储每个vocab[key]中的关键词，将各个关键词的值初始化为0是为了方便之后的统计。

In [7]:
%%time
stp = stopwords.words('english')
stm = ss('english')
keys_labels=list(labels.keys())


symbols= [',','.',':','_','!','?','/','\'','\"','*','>','<','@','~','-','(',')','%','=','\\','^'
     ,'&','|','#','$','0','1','2','3','4','5','6','7','8','9','10','[',']','+','{','}',';','`','~']

vocab=[[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]  #用来存放每个class文档所含的词
vocab_deputy=[{},{},{},{},{},                                           #用来存储每个class文档的关键词
    {},{},{},{},{},
    {},{},{},{},{}, 
    {},{},{},{},{},]
for key in keys_labels:
    text_path=[]
    wordlist={}
    for path0 in os.listdir(labels[key]):
        path1=labels[key]+"/"+path0
        text_path.append(path1)
        
    for lane in text_path:
        d = open(lane,'rb')
        data = d.read()
        encode = chardet.detect(data)['encoding']
        with codecs.open(lane, encoding=encode) as d:
            words = d.read()
            for symbol in symbols:
                 words = words.replace(symbol,'')
            words = words.split()
            for word in words:
                word = stm.stem(word)
                if word not in stp:
                    if word in wordlist:
                        wordlist[word]+=1
                    else:
                        wordlist[word]=1
            
    for i in wordlist:
        if wordlist[i] >= 4 and wordlist[i] <= 1000:  #筛掉词频过低及过高的词
            vocab[key].append("%s" %i)
            vocab_deputy[key][i]=0                    #将每个class关键词存入vocab_deputy中

Wall time: 1min 12s


In [8]:
print(list(vocab_deputy[0].keys())[:100]) #第一类文档提取的前100个关键词

['mathew', 'mathewmantiscouk', 'subject', 'altath', 'faq', 'atheist', 'resourc', 'archivenam', 'altatheismarchivenam', 'lastmodifi', 'version', 'address', 'organ', 'usa', 'freedom', 'religion', 'foundat', 'darwin', 'fish', 'assort', 'avail', 'us', 'po', 'box', 'wi', 'telephon', 'evolut', 'design', 'sell', 'symbol', 'like', 'christian', 'stick', 'car', 'feet', 'word', 'written', 'insid', 'north', 'hollywood', 'ca', 'peopl', 'san', 'bay', 'area', 'get', 'lynn', 'gold', 'tri', 'mail', 'net', 'go', 'direct', 'price', 'per', 'american', 'press', 'aap', 'publish', 'various', 'book', 'critiqu', 'bibl', 'list', 'biblic', 'contradict', 'handbook', 'wp', 'ball', 'foot', 'pp', 'isbn', 'nd', 'edit', 'absurd', 'immor', 'contain', 'base', 'king', 'jame', 'austin', 'tx', 'road', 'fax', 'prometheus', 'includ', 'haught', 'holi', 'horror', 'see', 'east', 'street', 'buffalo', 'new', 'york', 'altern', 'may', 'newer', 'older', 'drive']


In [9]:
print(type(vocab[0]))

<class 'list'>


## 基于贝叶斯概率计算构建模型

['daniel', 'subject', 'religion', 'homosexu', 'comput', 'write', 'recent', 'realiz', 'also', 'return', 'good', 'friend', 'point', 'sin', 'bibl', 'well', 'dont', 'see', 'could', 'consid', 'first', 'far', 'know', 'onli', 'male', 'explicit', 'mention', 'hook', 'think', 'ani', 'event', 'plenti', 'peopl', 'mani', 'denomin', 'person', 'sexual', 'identif', 'immor', 'lifestyl', 'choic', 'alway', 'somewhat', 'liber', 'prochoic', 'seem', 'religi', 'mix', 'either', 'told', 'anoth', 'misconcept', 'whole', 'stori', 'former', 'minist', 'lesbian', 'profession', 'sever', 'open', 'gay', 'like', 'go', 'look', 'accept', 'view', 'howev', 'us', 'believ', 'spiritu', 'freedom', 'conscienc', 'luck', 'journey', 'theolog', 'school', 'univers', 'divin', 'pharveyquackkfucom', 'paul', 'harvey', 'apr', 'promis', 'psalm', 'articl', 'psyrobtwubvmsbccbuffaloedu', 'robert', 'weiss', 'hear', 'lord', 'speak', 'peac', 'unto', 'saint', 'let', 'turn', 'folli', 'leader', 'favor', 'land', 'restor', 'fortun', 'forgiv', 'iniqu'

In [10]:
vocab_specialword_prob =  [
    {},{},{},{},{}, 
    {},{},{},{},{},
    {},{},{},{},{}, 
    {},{},{},{},{},
              ] #用来存储每个class中每个词所对应的“有该词的文档”在该类中的概率

此处同样是对每个class下的每个文档文件进行操作，将每个文档中经筛选后剩下的词构成一个集合，因为集合元素的互异性，所以那些在一个文档中出现多次的词在集合中也就只是一个元素了，这是方便计算每个class所含有的总词数的，即是为了满足“每个词在其出现的文档中只计算一次”。接下来只要每个文档的集合中有元素能在之前所构建的对应vocab[key]中找到，那么vocab_deputy[key]中相应位置的值就加一。同时，该类下每个文档的关键词都被提取组成一个样本存入样本集test_before。（最终该样本集将会有18828个样本） 在以上过程中每个样本集关键词的个数也被计算并在其对应的class下求和，其所构建的len_label中将存有20个class的每个class的总词数（每个词在出现的文档中只计算一次）。

In [11]:

%%time
len_label=[]
test_before=
for key in keys_labels:
    dic_keys=list(vocab_deputy[key].keys())
    text_path2=[]
    num = 0
    for path0 in os.listdir(labels[key]):
        path1=labels[key]+"/"+path0
        text_path2.append(path1)
    
    for lane in text_path2:
        space=[]
        f = open(lane,'rb')
        data = f.read()
        encode = chardet.detect(data)['encoding']
        with codecs.open(lane, encoding=encode) as f:
            lines=f.read()
            for symbol in symbols:
                 lines = lines.replace(symbol,'')
            lines = lines.split()
            lines = [line.strip('\n') for line in lines]
            lines=set(lines)  #集合元素具有互异性
            for item in lines:
                item = stm.stem(item)
                if item in vocab[key]:
                    vocab_deputy[key][item]+=1   #用来存储每个class中所含有的总词数（每个词在其出现的文档中只在该文档处计算一次）
                    space.append(item)
            num+=len(set(space))
            space.insert(0,key)                  #在每个样本首位插入该样本所对应的class
            test_before.append(space)            #存入每个文档的词袋以构造样本集
            
    len_label.append(num)                 

Wall time: 2min 1s




这里随机打乱样本集，方便以后抽取部分作为测试。

In [12]:
import random
random.shuffle(test_before)   #打乱样本集

In [13]:

print( vocab_deputy[1])

{'lipmanoasysdtnavymil': 2, 'robert': 51, 'lipman': 2, 'call': 137, 'present': 70, 'navi': 6, 'seminar': 10, 'scientif': 29, 'visual': 79, 'virtual': 47, 'realiti': 42, 'tuesday': 6, 'june': 9, 'carderock': 4, 'divis': 13, 'naval': 6, 'surfac': 79, 'warfar': 5, 'center': 60, 'former': 8, 'david': 52, 'taylor': 3, 'research': 83, 'bethesda': 4, 'maryland': 7, 'sponsor': 8, 'engin': 62, 'softwar': 186, 'system': 215, 'purpos': 43, 'exchang': 21, 'inform': 177, 'navyrel': 2, 'program': 349, 'develop': 94, 'applic': 99, 'aspect': 22, 'current': 110, 'work': 254, 'propos': 25, 'organ': 43, 'consid': 34, 'four': 26, 'type': 101, 'avail': 167, 'regular': 31, 'minut': 24, 'length': 14, 'short': 42, 'video': 114, 'standalon': 10, 'videotap': 7, 'author': 63, 'need': 348, 'demonstr': 20, 'accept': 41, 'publish': 45, 'ani': 455, 'proceed': 19, 'howev': 69, 'materi': 41, 'attende': 6, 'abstract': 15, 'submit': 18, 'one': 295, 'page': 55, 'andor': 29, 'code': 149, 'voic': 43, 'fax': 126, 'email': 2

In [14]:
print(len_label)   #每个class文档中出现的词所对应的出现文档数之和（一个词在一个文档中只计算一次）
print(sum(len_label))

[78527, 65610, 56248, 57870, 54307, 65629, 44982, 70541, 69019, 73651, 80956, 98998, 63750, 89041, 89660, 109371, 98511, 132383, 92505, 63632]
1555191


In [15]:
D={}    
for key in keys_labels:    
    D.update(vocab_deputy[key])   #合并20个字典

In [16]:
print(test_before[0])

[12, 'rubincisohiostateedu', 'hi', 'line', 'work', 'express', 'love', 'purchas', 'buck', 'style', 'great', 'number', 'dan', 'profession', 'go', 'control', 'thanx', 'j', 'possibl', 'ohio', 'joseph', 'assist', 'call', 'rubin', 'advanc', 'tri', 'part', 'mani', 'ask', 'joystick', 'free', 'project', 'augment', 'implement', 'game', 'button', 'arcad', 'daniel', 'anyon', 'tell', 'would', 'etc', 'also', 'found', 'dayton', 'could', 'part', 'complet']


In [25]:
DD=list(D.keys())    #包含20类文档所有关键词的list
print(DD[:100])

['mathew', 'mathewmantiscouk', 'subject', 'altath', 'faq', 'atheist', 'resourc', 'archivenam', 'altatheismarchivenam', 'lastmodifi', 'version', 'address', 'organ', 'usa', 'freedom', 'religion', 'foundat', 'darwin', 'fish', 'assort', 'avail', 'us', 'po', 'box', 'wi', 'telephon', 'evolut', 'design', 'sell', 'symbol', 'like', 'christian', 'stick', 'car', 'feet', 'word', 'written', 'insid', 'north', 'hollywood', 'ca', 'peopl', 'san', 'bay', 'area', 'get', 'lynn', 'gold', 'tri', 'mail', 'net', 'go', 'direct', 'price', 'per', 'american', 'press', 'aap', 'publish', 'various', 'book', 'critiqu', 'bibl', 'list', 'biblic', 'contradict', 'handbook', 'wp', 'ball', 'foot', 'pp', 'isbn', 'nd', 'edit', 'absurd', 'immor', 'contain', 'base', 'king', 'jame', 'austin', 'tx', 'road', 'fax', 'prometheus', 'includ', 'haught', 'holi', 'horror', 'see', 'east', 'street', 'buffalo', 'new', 'york', 'altern', 'may', 'newer', 'older', 'drive']


In [18]:
#此处字典初始化很重要，防止下面计算概率时出现key error
for k in vocab_specialword_prob:
    for j in DD:
        k[j]=0

for key in keys_labels:
    for j in DD:
        if j not in vocab_deputy[key]:
            vocab_deputy[key][j]=0

下面计算每个词在每个class下出现的概率，假定每个词的出现都是独立的，那么一个样本在该class下的概率将是其所含各个词在该class下概率的乘积，这样的话其值必然会很低，因此此处我们引入对数函数log，将概率计算由相乘变为相加。

In [29]:
%%time
import math
for j in DD:
    for key in keys_labels:
        vocab_specialword_prob[key][j]=math.log((vocab_deputy[key][j]+1)/(len_label[key]+2),2)  #计算每个词在每个class下出现的概率
        


Wall time: 402 ms


In [30]:
vocab_specialword_prob[2][DD[5]]

-15.779565475879126

## 测试构建的模型

观察该样本在哪个class下的概率最大，就判断样本属于哪个class。然后将每个样本的判断结果和它的真实所属class进行比较，计算出由随机2000个样本所组测试集来测试的贝叶斯分类器的准确率。

In [31]:
predict=[]
result=[]
test=test_before[:2000]  #从随机样本集里取2000个样本
X=[T[1:] for T in test]  
Y=[T[0] for T in test]  #存放2000个样本的标签


In [32]:
for x in X:
    prob=np.zeros(20)
    for key in keys_labels:
        for item in x:
            prob[key]+=vocab_specialword_prob[key][item]  #采用了log，固用加法计算
    predict.append(np.argmax(prob))   #存储各个样本最大概率所属的class
    

In [33]:
for n in range(len(X)):
    if predict[n]==Y[n]:
        result.append(1)
    else:
        result.append(0)        

In [34]:
print('accuracy==',sum(result)/2000*100,'%')

accuracy== 96.5 %
